Thie file contains the code to reading data in Spark for parallel processing

1) importing the dependencies <br>
driver memory is set to allocate more memory to spark application <br>
when running in standalone mode, memory has to be provided

In [ ]:
import os
import sys
import pandas as pd
import numpy as np

# Path for spark source folder
os.environ['SPARK_HOME']="/home/ashutosh/Dropbox/spark-2.1.1-bin-hadoop2.7"
os.environ['PYSPARK_SUBMIT_ARGS'] = ' --driver-memory 50g pyspark-shell'

# Append pyspark  to Python Path
sys.path.append("/home/ashutosh/Dropbox/spark-2.1.1-bin-hadoop2.7/python")
sys.path.append("/home/ashutosh/Dropbox/spark-2.1.1-bin-hadoop2.7/python/lib/py4j-0.10.4-src.zip")

2) Initializing Spark and Spark context

In [ ]:
try:
    from pyspark import SparkContext
    from pyspark import SparkConf
    from pyspark.sql import SparkSession

except ImportError as e:
    sys.exit(1)

conf = SparkConf().setMaster("local[*]").setAppName("My App")
sc = SparkContext(conf=conf)
spark = SparkSession.builder.getOrCreate()

3) Spark calculating the number of times a product is ordered by each user<br>
parse1: x[5] is the product_id and x[6] is the user_id<br>
parse2: returning it in the form easy to read in pandas dataframe

In [ ]:
temp = sc.textFile("filename.csv")
def parse1(x):
    x = str(x)
    x = x.split(",")
    return ((int(x[6]),int(x[5])),1)
def parse2(x):
    return (x[0][0],x[0][1],x[1])
temp = temp.filter(lambda x: "order_id" not in x).map(parse1).reduceByKey(lambda x,y: x+y).map(parse2)
temp = temp.collect()

4) Developing a dataframe from the sc

In [ ]:
label = ['user_id','product_id','count']
temp = pd.DataFrame.from_records(temp,columns=label)
temp.to_csv("filename.csv")

Another example of using the Spark calculation

In this example, a numpy array of orders is made<br>
where the input is +1 when order for a product observed

parse1: is used to find which products are ordered in each order<br>
parse2: is used to sort the customer orders in their order number (visits)<br>
parse3: is used to sum the orders to find out the frequency of each order<br>

The it is converted into a dataframe

In [ ]:
working_data_format = sc.textFile("working_data_format.csv")

def parse1(x):
    x = x.split(",")
    y = np.array([0]*len(products))
    y[products.index(int(x[5]))] += 1
    return ((int(x[6]),int(x[7])),y) # user_id, order_number, y

def parse2(x):
    return (x[0][1],(x[0][0],x[1].tolist())) # order_number, user_id, y

def parse3(x):
    return (x[1][0],x[1][1],x[0])

working_data = working_data.filter(lambda x: "order_id" not in x and int(x.split(",")[4]) in products).map(parse1).reduceByKey(lambda  x,y: x+y).map(parse2).sortByKey().map(parse3).reduceByKey(lambda x,y:x+y)

working_data_format = working_data.collect()
label = ['user_id','input','order_number']
working_data_format = pd.DataFrame.from_records(working_data_format,columns=label)
working_data_format.to_csv("working_data_format.csv")